In [1]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the datasets
customers_df = pd.read_csv('Customers.csv')
products_df = pd.read_csv('Products.csv')
transactions_df = pd.read_csv('Transactions.csv')

In [2]:
customers_df['SignupDate'] = pd.to_datetime(customers_df['SignupDate'])
transactions_df['TransactionDate'] = pd.to_datetime(transactions_df['TransactionDate'])


In [3]:
merged_data = transactions_df.merge(customers_df, on='CustomerID', how='left')
merged_data = merged_data.merge(products_df, on='ProductID', how='left')


In [4]:
merged_data = merged_data.drop(columns=['Price_x'])  # Keep only one Price column


# Sales Analysis

In [8]:
sales_by_region = merged_data.groupby('Region')['TotalValue'].sum().sort_values(ascending=False)
print(sales_by_region)


Region
South America    219352.56
Europe           166254.63
North America    152313.40
Asia             152074.97
Name: TotalValue, dtype: float64


In [6]:
top_products = merged_data.groupby('ProductName')['TotalValue'].sum().sort_values(ascending=False).head(5)
print(top_products)


ProductName
ActiveWear Smartwatch    39096.97
SoundWave Headphones     25211.64
SoundWave Novel          24507.90
ActiveWear Jacket        22712.56
ActiveWear Rug           22314.43
Name: TotalValue, dtype: float64


In [7]:
avg_customer_spending = merged_data.groupby('CustomerID')['TotalValue'].sum().mean()
print(avg_customer_spending)


3467.3143718592933


# Time Trends

In [9]:
merged_data['Month'] = merged_data['TransactionDate'].dt.to_period('M')
monthly_sales = merged_data.groupby('Month')['TotalValue'].sum()
print(monthly_sales)


Month
2023-12     3769.52
2024-01    66376.39
2024-02    51459.27
2024-03    47828.73
2024-04    57519.06
2024-05    64527.74
2024-06    48771.18
2024-07    71366.39
2024-08    63436.74
2024-09    70603.75
2024-10    47063.22
2024-11    38224.37
2024-12    59049.20
Freq: M, Name: TotalValue, dtype: float64


In [10]:
daily_sales = merged_data.groupby(merged_data['TransactionDate'].dt.date)['TotalValue'].sum()
print(daily_sales.sort_values(ascending=False).head(5))


TransactionDate
2024-09-04    7585.50
2024-12-18    6843.94
2024-12-03    6258.32
2024-07-14    5880.49
2024-05-08    5813.44
Name: TotalValue, dtype: float64


# Product and Category Analysis 

In [11]:
category_sales = merged_data.groupby('Category')['TotalValue'].sum().sort_values(ascending=False)
print(category_sales)


Category
Books          192147.47
Electronics    180783.50
Clothing       166170.66
Home Decor     150893.93
Name: TotalValue, dtype: float64


In [12]:
product_quantity = merged_data.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False)
print(product_quantity)


ProductName
ActiveWear Smartwatch       100
SoundWave Headphones         97
HomeSense Desk Lamp          81
ActiveWear Rug               79
SoundWave Cookbook           78
                           ... 
HomeSense Headphones         18
SoundWave Laptop             16
SoundWave Jacket             16
BookWorld Wall Art           15
ComfortLiving Headphones     15
Name: Quantity, Length: 66, dtype: int64


# Customer Insights

In [13]:
top_customers = merged_data.groupby('CustomerID')['TotalValue'].sum().sort_values(ascending=False).head(5)
print(top_customers)


CustomerID
C0141    10673.87
C0054     8040.39
C0065     7663.70
C0156     7634.45
C0082     7572.91
Name: TotalValue, dtype: float64


# Customer Retention Insight

In [14]:
# Calculate customer tenure (days since signup)
merged_data['CustomerTenure'] = (merged_data['TransactionDate'] - merged_data['SignupDate']).dt.days

# Average revenue per customer tenure group (e.g., <30 days, 30-90 days, >90 days)
bins = [0, 30, 90, 180, 365, merged_data['CustomerTenure'].max()]
labels = ['<30 Days', '30-90 Days', '90-180 Days', '180-365 Days', '>365 Days']
merged_data['TenureGroup'] = pd.cut(merged_data['CustomerTenure'], bins=bins, labels=labels)

tenure_sales = merged_data.groupby('TenureGroup')['TotalValue'].sum().sort_values(ascending=False)
print("\nRevenue by Customer Tenure Group:")
print(tenure_sales)



Revenue by Customer Tenure Group:
TenureGroup
>365 Days       340178.59
180-365 Days     86434.86
90-180 Days      58505.55
30-90 Days       39131.35
<30 Days         24259.20
Name: TotalValue, dtype: float64
